In [ ]:
import VideoMamba as vm

In [ ]:
class VideoMambaPose(nn.Module):
    def super().__init__(self):
        self.encoder = vm.videomamba_small() # TODO this is temporary 

    def forward(self, x):
        x = self.encoder(x)

        # adding my own layer, see how deciwatch did it.
        x = 


: 